In [1]:
%load_ext autoreload
%autoreload 2

%pylab inline

import dataloaders
import models
import sklearn.linear_model as linear_model
import sklearn.metrics as metrics 
import sklearn.cluster as cluster
import numpy as np 
import random
import classifiers
from torch.autograd import Variable
import torch
import torch.nn as nn 
import torch.optim as optim

from imblearn.under_sampling import RandomUnderSampler

Populating the interactive namespace from numpy and matplotlib


In [2]:
filepath = '../data/soc-sign-epinions-cleaned.csv'
data = dataloaders.UnsplitDataset(filepath, ratio=0.8, delimiter=',')

In [3]:
num_nodes = data.get_num_nodes()
dims = 32
epochs = 100
lr = 0.1
lr_decay=0.0
weight_decay=0.0
lam = 0.00055
p = 2
delta = 1
delta0 = 0.5
dims_array = [dims, 20, 20]

In [4]:
triples, triples0 = data.get_training_triples(True)

In [ ]:
batch_size = int(0.4 * len(triples))
batch_size0 = int(0.4 * len(triples0))

In [ ]:
sine_model = models.fit_sine_model(num_nodes, dims_array, triples, triples0, delta, delta0,
                                   batch_size, batch_size0, epochs, lr=lr, lr_decay=lr_decay,
                                  lam=lam, p=p)

Loss at epoch  1  is  2.141263008117676
Loss at epoch  2  is  1.8648185729980469
Loss at epoch  3  is  1.4938678741455078
Loss at epoch  4  is  1.3047062158584595
Loss at epoch  5  is  1.1976044178009033
Loss at epoch  6  is  1.1234960556030273
Loss at epoch  7  is  1.068747878074646
Loss at epoch  8  is  1.028942346572876
Loss at epoch  9  is  0.995143473148346
Loss at epoch  10  is  0.9652245044708252
Loss at epoch  11  is  0.9396826028823853
Loss at epoch  12  is  0.9175252318382263
Loss at epoch  13  is  0.8954623937606812
Loss at epoch  14  is  0.875285804271698
Loss at epoch  15  is  0.8571527600288391
Loss at epoch  16  is  0.8418391942977905
Loss at epoch  17  is  0.8284574747085571
Loss at epoch  18  is  0.8112876415252686
Loss at epoch  19  is  0.7933687567710876
Loss at epoch  20  is  0.7775708436965942
Loss at epoch  21  is  0.7658843398094177
Loss at epoch  22  is  0.7567294836044312
Loss at epoch  23  is  0.7509208917617798
Loss at epoch  24  is  0.7319033145904541
Loss a

In [9]:
for i, param in enumerate(sine_model.parameters()):
    print(i)

0
1
2
3
4
5
6
7


In [10]:
X_train, y_train = data.get_training_set()
X_test, y_test = data.get_testing_set()

In [11]:
clf = linear_model.LogisticRegression()
reports = classifiers.train_and_evaluate_classifier(clf, sine_model, X_train, y_train, X_test, y_test,
                                                    operation='l2', undersample=True, ratio=3)

Assembling training set features....
Fitting classifier model
Assembling testing set features


In [12]:
reports

{'auc': 0.63556034467045697,
 'average_percision_score': 0.84457887770899254,
 'classification_report': '             precision    recall  f1-score   support\n\n          0       0.44      0.04      0.08     24153\n          1       0.77      0.98      0.86     79162\n\navg / total       0.69      0.76      0.68    103315\n',
 'confusion_matrix': array([[ 1027, 23126],
        [ 1324, 77838]]),
 'macro_f1': 0.47087970983949434,
 'micro_f1': 0.76334510961622226}

In [13]:
print(reports['auc'])

0.63556034467


In [14]:
print(reports['micro_f1'])

0.763345109616


In [15]:
print(reports['classification_report'])

             precision    recall  f1-score   support

          0       0.44      0.04      0.08     24153
          1       0.77      0.98      0.86     79162

avg / total       0.69      0.76      0.68    103315



In [16]:
print(reports['confusion_matrix'])

[[ 1027 23126]
 [ 1324 77838]]


In [19]:
from scipy.spatial import distance
dist_met = distance.cosine
pos_distances = []
for x, y in X_train[y_train == 1]:
    x_emb = sine_model.get_embedding(x)
    y_emb = sine_model.get_embedding(y)
    pos_distances.append(dist_met(x_emb, y_emb))
    
neg_distances = []
for x, y in X_train[y_train == 0]:
    x_emb = sine_model.get_embedding(x)
    y_emb = sine_model.get_embedding(y)
    neg_distances.append(dist_met(x_emb, y_emb))
    
print(np.mean(pos_distances))
print(np.mean(neg_distances))
print(np.std(pos_distances))
print(np.std(neg_distances))


pos_distances1 = []
for x, y in X_test[y_test == 1]:
    x_emb = sine_model.get_embedding(x)
    y_emb = sine_model.get_embedding(y)
    pos_distances1.append(dist_met(x_emb, y_emb))
    
neg_distances1 = []
for x, y in X_test[y_test == 0]:
    x_emb = sine_model.get_embedding(x)
    y_emb = sine_model.get_embedding(y)
    neg_distances1.append(dist_met(x_emb, y_emb))
    
print(np.mean(pos_distances1))
print(np.mean(neg_distances1))

print(np.std(pos_distances1))
print(np.std(neg_distances1))

0.999379532508
1.01730525188
0.179853851779
0.17695632218
1.00052123149
1.01069444434
0.179554389231
0.176938804426


In [88]:
len(y_test[y_test == 0])/len(y_test)

0.23493200406523737

In [24]:
len(y_train[y_train == 0])/len(y_train)

0.23211779509267774

In [25]:
y = data.y

In [251]:
len(y[y == 0]) / len(y)

0.10010676556529557

In [252]:
count = 0.0
for x, y in X_test[y_test == 0]:
    x = Variable(torch.LongTensor([int(x)]))
    y = Variable(torch.LongTensor([int(y)]))
    output = kernel_model(x, y).data.numpy()[0]
    neg = output[0]
    pos = output[1]
    count += 1 if neg > pos else 0
print(count / len(X_test[y_test == 0]))

0.8798882681564246


In [253]:
count = 0.0
for x, y in X_test[y_test == 1]:
    x = Variable(torch.LongTensor([int(x)]))
    y = Variable(torch.LongTensor([int(y)]))
    output = kernel_model(x, y).data.numpy()[0]
    neg = output[0]
    pos = output[1]
    count += 1 if neg < pos else 0
print(count / len(X_test[y_test == 1]))

0.8204247345409119


In [254]:
count = 0.0
A = []
b = []
for x, y in X_test[y_test == 0]:
    x = Variable(torch.LongTensor([int(x)]))
    y = Variable(torch.LongTensor([int(y)]))
    output = kernel_model(x, y).data.numpy()[0]
    neg = output[0]
    pos = output[1]
    A.append(pos)
    b.append(0)
    count += 1 if neg > pos else 0
print(count / len(X_test[y_test == 0]))
count = 0.0
for x, y in X_test[y_test == 1]:
    x = Variable(torch.LongTensor([int(x)]))
    y = Variable(torch.LongTensor([int(y)]))
    output = kernel_model(x, y).data.numpy()[0]
    neg = output[0]
    pos = output[1]
    A.append(pos)
    b.append(1)
    count += 1 if neg < pos else 0
print(count / len(X_test[y_test == 1]))

0.8798882681564246
0.8204247345409119


In [255]:
metrics.roc_auc_score(b, A)

0.91300304628043227

In [256]:
class EnergyToProbsLayer(nn.Module):
    def __init__(self):
        super(EnergyToProbsLayer, self).__init__()
        self.transform = nn.Sigmoid()
    def forward(self, x):
        ones = torch.ones_like(x)
        positive_prob = self.transform(x)
        negative_prob = ones - positive_prob
        output = torch.cat((negative_prob, positive_prob), dim=1)
        #output = torch.log(output)
        return output
        

class BilinearEmbeddingModel(nn.Module):
    def __init__(self, num_nodes, dims=128):
        super(BilinearEmbeddingModel, self).__init__()
        self.embeddings = nn.Embedding(num_nodes + 1, dims)
        self.initial_layer = nn.Linear(dims, int(dims/2))
        self.initial_transform = nn.ReLU()
        self.layer = nn.Bilinear(int(dims/2), int(dims/2), 1)
        self.transform_layer= EnergyToProbsLayer()
    
    def get_embedding(self, x):
        x = Variable(torch.LongTensor([x]))
        emb = self.embeddings(x)
        transformed = self.initial_layer(emb)
        emb = transformed.data.numpy()[0]
        return emb

        
    def forward(self, u, v):
        u_emb = self.embeddings(u)
        v_emb = self.embeddings(v)
        u_emb = self.initial_transform(self.initial_layer(u_emb))
        v_emb = self.initial_transform(self.initial_layer(v_emb))
        energy = self.layer(u_emb, v_emb)
        probs = self.transform_layer(energy)
        return probs

In [257]:
X_train_pos = X_train[y_train == 1]
X_train_neg = X_train[y_train == 0]

In [258]:
len(X_train_neg) - len(X_train_pos)

-25622

In [259]:
batch_frac = 0.1
epochs = 200
alpha = 0.7
dim = 32
beta = 0.1
size = data.get_num_nodes()
print(size)
model = BilinearEmbeddingModel(size, dim)
optimizer = optim.Adagrad(model.parameters(), lr=alpha, lr_decay=beta)
lam = 0.0055

5881


In [260]:
negative_batch_size = len(X_train_neg)
positive_batch_size = int(0.8 * negative_batch_size)
num_positive = len(X_train_pos)


criterion = nn.CrossEntropyLoss(size_average=True)
regularizer = lambda x: torch.norm(x - torch.zeros_like(x))
print(negative_batch_size)
print(positive_batch_size)

3205
2564


In [261]:
for epoch in range(epochs):
    model.zero_grad()

    negative_batch = X_train_neg
    positive_batch_idx = random.sample(range(num_positive), positive_batch_size)
    positive_batch = X_train_pos[positive_batch_idx,:]
    
    
    actual = [0] * negative_batch_size
    actual1 = [1] * positive_batch_size
    actual.extend(actual1)
    actual = np.array(actual)
    
    
    all_inputs = np.concatenate((negative_batch, positive_batch), axis=0)
    us = all_inputs[:,0]
    vs = all_inputs[:,1]
    
    us = Variable(torch.LongTensor(us))
    vs = Variable(torch.LongTensor(vs))
    actual = Variable(torch.LongTensor(actual))
    
    #print('Computing loss....')
    probs = model(us, vs)
    #print('Number of probs ', len(probs))
    #print(probs)
    #print('Number of examples ', len(actual))
    loss = criterion(probs, actual)
    for parameter in model.parameters():
        loss += lam * regularizer(parameter)
        
        
    
    
    print('Loss at iteration ', epoch, ' is ', loss.data)
    loss.backward()
    optimizer.step()

Loss at iteration  0  is  
 3.1082
[torch.FloatTensor of size 1]

Loss at iteration  1  is  
 2.4626
[torch.FloatTensor of size 1]

Loss at iteration  2  is  
 1.7610
[torch.FloatTensor of size 1]

Loss at iteration  3  is  
 1.3609
[torch.FloatTensor of size 1]

Loss at iteration  4  is  
 1.4519
[torch.FloatTensor of size 1]

Loss at iteration  5  is  
 1.2471
[torch.FloatTensor of size 1]

Loss at iteration  6  is  
 1.1656
[torch.FloatTensor of size 1]

Loss at iteration  7  is  
 1.0918
[torch.FloatTensor of size 1]

Loss at iteration  8  is  
 1.0441
[torch.FloatTensor of size 1]

Loss at iteration  9  is  
 0.9981
[torch.FloatTensor of size 1]

Loss at iteration  10  is  
 0.9836
[torch.FloatTensor of size 1]

Loss at iteration  11  is  
 0.9513
[torch.FloatTensor of size 1]

Loss at iteration  12  is  
 0.9050
[torch.FloatTensor of size 1]

Loss at iteration  13  is  
 0.8665
[torch.FloatTensor of size 1]

Loss at iteration  14  is  
 0.8369
[torch.FloatTensor of size 1]

Loss 

Loss at iteration  126  is  
 0.4825
[torch.FloatTensor of size 1]

Loss at iteration  127  is  
 0.4871
[torch.FloatTensor of size 1]

Loss at iteration  128  is  
 0.4828
[torch.FloatTensor of size 1]

Loss at iteration  129  is  
 0.4860
[torch.FloatTensor of size 1]

Loss at iteration  130  is  
 0.4814
[torch.FloatTensor of size 1]

Loss at iteration  131  is  
 0.4851
[torch.FloatTensor of size 1]

Loss at iteration  132  is  
 0.4836
[torch.FloatTensor of size 1]

Loss at iteration  133  is  
 0.4808
[torch.FloatTensor of size 1]

Loss at iteration  134  is  
 0.4819
[torch.FloatTensor of size 1]

Loss at iteration  135  is  
 0.4827
[torch.FloatTensor of size 1]

Loss at iteration  136  is  
 0.4815
[torch.FloatTensor of size 1]

Loss at iteration  137  is  
 0.4833
[torch.FloatTensor of size 1]

Loss at iteration  138  is  
 0.4833
[torch.FloatTensor of size 1]

Loss at iteration  139  is  
 0.4823
[torch.FloatTensor of size 1]

Loss at iteration  140  is  
 0.4820
[torch.Floa

In [262]:

classifier = linear_model.LogisticRegression()

In [263]:
def embedding(model, x):
    x_var = Variable(torch.LongTensor([x]))
    emb = model.embeddings(x_var)
    emb = model.initial_layer(emb)
    emb = emb.data.numpy()[0]
    return emb
    

def get_hadamard(model, x, y):
    x_features = embedding(model, x)
    y_features = embedding(model, y)
    return x_features * y_features

In [264]:

feature_extractor = get_hadamard

In [265]:
inputs = []
outputs = []
for x, y in X_train_neg:
    x = int(x)
    y = int(y)
    inputs.append(feature_extractor(model, x, y))
    outputs.append(0)

positive_batch_idx = random.sample(range(num_positive), positive_batch_size)
positive_batch = X_train_pos[positive_batch_idx,:]
for x, y in positive_batch:
    x = int(x)
    y = int(y)
    inputs.append(feature_extractor(model, x, y))
    outputs.append(1)

inputs = np.array(inputs)
outputs = np.array(outputs)

In [266]:
classifier.fit(inputs, outputs)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [267]:
inputs1 = []
expected_outputs1 = []

count = 0

for x, y in X_test[y_test == 0]:
    x = int(x)
    y = int(y)
    inputs1.append(feature_extractor(model, x, y))
    expected_outputs1.append(0)
    count += 1
print(count)

for x, y in X_test[y_test == 1]:
    x = int(x)
    y = int(y)
    inputs1.append(feature_extractor(model, x, y))
    expected_outputs1.append(1)
    count += 1
print(count)

inputs1 = np.array(inputs1)
expected_outputs1 = np.array(expected_outputs1)
print(len(expected_outputs1))
actual_outputs1 = classifier.predict(inputs1)
actual_probs1 = classifier.predict_proba(inputs1)[:,1]


358
3560
3560


In [268]:
metrics.f1_score(expected_outputs1, actual_outputs1, average='macro')

0.75784200322129891

In [269]:
metrics.roc_auc_score(expected_outputs1, actual_probs1)

0.88987068138279501

In [270]:
print(metrics.classification_report(expected_outputs1, actual_outputs1))

             precision    recall  f1-score   support

          0       0.47      0.77      0.58       358
          1       0.97      0.90      0.94      3202

avg / total       0.92      0.89      0.90      3560



In [271]:
len(expected_outputs1)

3560

In [272]:
len(X_test[y_test == 1]) + len(X_test[y_test == 0])

3560

In [273]:
metrics.confusion_matrix(expected_outputs1, actual_outputs1)

array([[ 275,   83],
       [ 315, 2887]])